In [ ]:
!pip install transformers

In [1]:
import tensorflow as tf
import json
import pandas as pd
import numpy as np
import os
import torch as t
import datetime
# import transformers
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: ignored

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", device=0) ## device = 0 forces it to use GPU

In [2]:
from google.colab import drive 
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/MyDrive/datax")
!ls

Mounted at /content/drive/
 01_reviews_filtering.ipynb   Model_v05_summ.ipynb
 austin_rest.csv	      rest_good_bad_review.csv
 austin_review.csv	      rest_with_good_bad_review.csv
'Data dump'		      sum_review_bus.csv
 example.csv		      zero_shot_test
 labeled


In [3]:
rev = pd.read_csv('austin_review.csv')
rest = pd.read_csv("austin_rest.csv")

In [4]:
still_open = rest[rest.is_open == 1]

In [5]:
still_open_rev = rev[rev.business_id.isin(still_open.business_id)]
still_open_rev.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4.0,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46
1,GgWFjRHhelaiUgR2-H6N4g,pRPT3vqhqpU7kHgmKJamvw,-_GnwXmzC3DXsHR9nyaC2g,3.0,0,0,0,3.5 stars! I got the avocado margarita and it ...,2012-11-06 07:09:57
4,z-6ElnK3yXpn2oNKR7NYtw,OSsqq7CzBdAbF7E6kZHd0g,bZiIIUcpgxh8mpKMDhdqbA,3.0,0,0,0,My husband and I shared the parmesan truffly f...,2017-10-11 00:25:47
5,nqUJoSkDr_1mqnLtOrWFSA,HLE4PMeOrkQkIR559IkY9A,gvpoFnaCq2J7TBXV7yFIkQ,5.0,0,0,0,"Perfect spices, nice atmosphere & music, great...",2017-07-28 19:17:54
6,h6o5mduBqDUkKsa4KRIDzw,Nw4GV60oM78Uizh9dUuqNQ,LUCmOKTK5Uh7eU84yJq3mQ,5.0,0,0,0,Snap is my go-to lunch spot nearly every day! ...,2015-06-08 18:07:06


In [6]:
## Select resturants that has more than 10 reviews and less than 500 reviews
df_rev_count = still_open_rev.groupby('business_id').agg({'review_id':'count'}).reset_index()
filtered_bid = df_rev_count[(df_rev_count.review_id >= 15) & (df_rev_count.review_id <= 500)]
rev_filtered_df = still_open_rev[still_open_rev.business_id.isin(filtered_bid.business_id)]

In [10]:
rev_filtered_df.shape

(301789, 9)

In [11]:
rev_filtered_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4.0,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46
6,h6o5mduBqDUkKsa4KRIDzw,Nw4GV60oM78Uizh9dUuqNQ,LUCmOKTK5Uh7eU84yJq3mQ,5.0,0,0,0,Snap is my go-to lunch spot nearly every day! ...,2015-06-08 18:07:06
7,D32k07EZNRF9C_f3F-eBMQ,GzCYPFBMk_j4LzmJpnVzQA,-L69Ix0-xX4BlHA61fGvrQ,5.0,3,1,2,This Chicken Express has been doing it right f...,2016-10-26 15:45:52
11,m0MvtD8fjZVPWgI3y_wxBA,wJHy7ZJG_EvLFQDRms5rXQ,75HV-KqCtn_oHeiLiGlO_w,4.0,0,0,0,Great place... delicious tapas and very nice w...,2013-10-24 02:29:10
19,Xx1BXanaoh9-NfHs0UA4Mw,0iTlg0tKm2toM139TFYVCg,IbuSlumTCNSD-ohMVHfWUA,4.0,0,0,0,Really tasty and very fresh food. We ordered a...,2018-09-14 02:08:52
...,...,...,...,...,...,...,...,...,...
807633,A_bdec6c70w_nH4ZqUu_UQ,ysUk0irSYQ-kZgAZoD5uSw,RmRAhZC0FcxlwW0nEZv1GQ,2.0,0,0,0,We have been going to Texican for years and si...,2020-11-29 16:27:22
807634,I_8BJ6vZo7mVHi4TzuEK2A,74bzP169TBBFLEnzQsCdHw,dgywJM4JpniwOEDNfrfewg,2.0,10,0,3,I rec'd my meal as part of a complimentary Yel...,2020-08-09 19:53:40
807636,SZwokVY7CHjgAgG8qCxebQ,nVN20tunCTGzxSXrhD9K6Q,O1pWzMvVmBomQUaZ3aUlLw,5.0,1,0,1,This place is legit! I had the Funky Monkey Fr...,2020-12-30 18:38:38
807638,UWHPbsQzfrmjtDCbq2AeDg,Cx85VtZCOwdzHATPlB6fyQ,Wn8j-1-ity6hdjP4bC_wkw,2.0,0,0,0,Terrible pho. Literally zero flavor. And the c...,2021-01-17 22:09:10


In [8]:
## filter rev less than 4
rev_filtered_df_less_4_star = rev_filtered_df[rev_filtered_df.stars < 4]

In [9]:
rev_filtered_df_less_4_star.reset_index(drop=True,inplace=True)

In [ ]:
expensive = ["expensive"]
bad_customer_service = ["bad customer service", "long wait time", "unfriendly"]
bad_ambience = ["bad ambience", "dirty", "crowded"]
small_portion = ["small portion"]
poor_food_quality = ["bad food taste", "overcooked", "undercooked", "bad drinks"]
all_category = expensive + bad_customer_service + bad_ambience + small_portion + poor_food_quality
all_category

['expensive',
 'bad customer service',
 'long wait time',
 'unfriendly',
 'bad ambience',
 'dirty',
 'crowded',
 'small portion',
 'bad food taste',
 'overcooked',
 'undercooked',
 'bad drinks']

In [ ]:
#Inputs
#df: dataframe with reviews (column name with reviews should be "text")
#tags: the tags you want to find in the reviews (Eg : ['expensive','costly','not worth the price'])
def zsl_processor(df, tags):
  
  summary_label = pd.DataFrame()

  for i in range(len(df)):
    rev_id = df['review_id'][i]
    msg = df['text'][i]
    bid = df['business_id'][i]
    result = classifier(msg, tags, multi_label=True)
    df_results = pd.DataFrame(result)
    
    for j in range(len(tags)):
      temp_rev_label = pd.DataFrame({
          'business_id':[bid], 
          'review_id':[rev_id], 
          'text':[msg], 
          'label':[df_results.labels[j]], 
          'score':[df_results.scores[j]],
          })
      
      summary_label = summary_label.append(temp_rev_label)

  return summary_label

In [ ]:
os.chdir('/content/drive/My Drive/datax/Data dump/')
n=1
for i in [0,10000,20000,30000,40000,50000,60000,70000,80000,90000]:
  print(n)
  n=n+1
  p=i
  if p==90000:
    q=len(rev_filtered_df_less_4_star)
  else:
    q=p+10000
  print(p,q)
  df=rev_filtered_df_less_4_star.copy().iloc[p:q,]
  df.reset_index(drop=True, inplace=True)
  start = datetime.datetime.now()
  test = zsl_processor(df, all_category) ## DONT FORGET TO CHANGE THIS 
  test = test.reset_index(drop=True)
  time_elapsed = datetime.datetime.now()
  name='test'+str(p)+'.csv'
  test.to_csv(name, index=False)


1
0 10000


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


2
10000 20000
3
20000 30000
4
30000 40000
5
40000 50000
6
50000 60000
7
60000 70000
8
70000 80000
9
80000 90000
10
90000 99242


In [ ]:
rev_filtered_df_less_4_star.shape


99242

In [ ]:
  df=rev_filtered_df_less_4_star.copy().iloc[10:20,]


In [ ]:
  test = zsl_processor(df, all_category) ## DONT FORGET TO CHANGE THIS 


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:910: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
